In [1]:
# !pip install wandb --user

     |████████████████████████████████| 1.8 MB 16.8 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 181 kB 89.1 MB/s            
     |████████████████████████████████| 284 kB 98.2 MB/s            
     |████████████████████████████████| 147 kB 97.0 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 63 kB 411 kB/s             
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21502 sha256=9f1781da4f563e1d159e151d5ba42011a6f4d44fe58dabb7107ad143e4600a1d
  Stored in directory: /home/ss4yd/.cache/pip/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=8e6b0a6d73b4a9ed0610afbee93676aa168eb32a3df3e562f77dddd44c3923d5
  Stored in directory: /home/ss4yd/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built promise pathtools
  Cons

In [1]:
import wandb

In [5]:
wandb.init(project="my-test-project")

wandb: Currently logged in as: ssen7. Use `wandb login --relogin` to force relogin


In [2]:
from transformers import BertTokenizer, AutoTokenizer, AutoModel

import pandas as pd
import numpy as np

import torch
import torch.nn as nn

#ref: https://mccormickml.com/2019/07/22/BERT-fine-tuning/

In [3]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [4]:
df = pd.read_pickle('../csv/generating_training_df.pickle')
df.columns = ['patch_paths', 'pid', 'cluster_assignment', 'complete_tokens','dtype', 'notes']
df.head()

,patch_paths,pid,cluster_assignment,complete_tokens,dtype,notes
0,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R55E-1726,4,"[2, pieces, 5x7, mm, broken, apart, good, morp...",train,2 pieces ~9.5x7 mm; 1 broken apart; good morph...
1,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R55E-1726,4,"[2, pieces, 5x7, mm, broken, apart, good, morp...",train,2 pieces ~9.5x7 mm; 1 broken apart; good morph...
2,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R55E-1726,4,"[2, pieces, 5x7, mm, broken, apart, good, morp...",train,2 pieces ~9.5x7 mm; 1 broken apart; good morph...
3,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R55E-1726,4,"[2, pieces, 5x7, mm, broken, apart, good, morp...",train,2 pieces ~9.5x7 mm; 1 broken apart; good morph...
4,/project/GutIntelligenceLab/ss4yd/gtex_data/pr...,GTEX-R55E-1726,4,"[2, pieces, 5x7, mm, broken, apart, good, morp...",train,2 pieces ~9.5x7 mm; 1 broken apart; good morph...


In [39]:
# token_dict = tokenizer.encode_plus()

In [5]:
max_len = 0

# For every sentence...
for sent in df.notes:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  75


In [10]:
def check_cuda():
    if torch.cuda.is_available():       
        device = torch.device("cuda")
        print(f'There are {torch.cuda.device_count()} GPU(s) available.')
        print('Device name:', torch.cuda.get_device_name(0))
        return device
    else:
        print('No GPU available, using the CPU instead.')
        device = torch.device("cpu")
        return device
    
device=check_cuda()

No GPU available, using the CPU instead.


In [6]:
token_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 80,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        truncation = True
                   )

In [7]:
token_dict['input_ids'][0]

print(tokenizer.convert_ids_to_tokens(token_dict['input_ids'][0]))

['[CLS]', '6', 'pieces', ';', 'mild', 'di', '##ff', '##use', 'inter', '##st', '##iti', '##al', 'fi', '##bro', '##sis', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [15]:
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [8]:
token_dict

{'input_ids': tensor([[  101,   127,  3423,   132, 10496,  4267,  3101,  5613,  9455,  2050,
         17030,  1348, 20497, 12725,  4863,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 